# Importing Packages

In [4]:
using Pkg

Pkg.add("Random")
Pkg.add("Statistics")
Pkg.add("DataFrames")
Pkg.add("CSV")
Pkg.add("Plots")

    Updating registry at `C:\Users\ultan\.julia\registries\General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Resolving package versions...
  No Changes to `C:\Users\ultan\.julia\environments\v1.6\Project.toml`
  No Changes to `C:\Users\ultan\.julia\environments\v1.6\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\ultan\.julia\environments\v1.6\Project.toml`
  No Changes to `C:\Users\ultan\.julia\environments\v1.6\Manifest.toml`
`DataFrames` is pinned at `v1.1.1`: maintaining pinned version
   Resolving package versions...
  No Changes to `C:\Users\ultan\.julia\environments\v1.6\Project.toml`
  No Changes to `C:\Users\ultan\.julia\environments\v1.6\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\ultan\.julia\environments\v1.6\Project.toml`
  No Changes to `C:\Users\ultan\.julia\environments\v1.6\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\ultan\.julia\environments\v1.6\Proj

In [37]:
using CSV, Tables
using DataFrames
using Statistics
using Random

# Import Data

In [31]:
stable_all_train_val_proc = CSV.read("train_val_processed.csv", DataFrame,header=true);

In [32]:
stable_y=stable_all_train_val_proc[:,4]

stable_x=select!(stable_all_train_val_proc,Not(:Item_Outlet_Sales))


,Item_Weight,Item_Visibility,Item_MRP,Years_Opened,Item_Type_Baking Goods,Item_Type_Breads
,Float64,Float64,Float64,Int64,Int64,Int64
1,14.6,0.0594052,179.698,16,0,0
2,16.25,0.025777,169.947,14,0,0
3,12.7929,0.0907781,153.102,28,0,0
4,13.85,0.0259367,164.921,14,0,0
5,9.195,0.0518271,77.4644,4,1,0
6,12.7929,-0.969538,52.6666,28,0,0
7,14.65,0.0721787,262.959,14,0,0
8,9.6,0.161182,165.316,15,0,0
9,16.85,0.159826,195.548,9,0,0


In [16]:
stable_y

8523-element Vector{Float64}:
 3735.138
  443.4228
 2097.27
  732.38
  994.7052
  556.6088
  343.5528
 4022.7636
 1076.5986
 4710.535
 1516.0266
 2187.153
 1589.2646
    ⋮
 4207.856
 2479.4392
  595.2252
  468.7232
 1571.288
  858.882
 3608.636
 2778.3834
  549.285
 1193.1136
 1845.5976
  765.67

In [20]:
#stable_x = CSV.read("stable_.csv", DataFrame,header=true);
#stable_y = CSV.read("train_processed_Y.csv", DataFrame,header=true);
#stableX_test = CSV.read("test_processed_without_Y.csv", DataFrame,header=false);
#stableY_test = CSV.read("test_processed_Y.csv", DataFrame,header=false);

# Accuracy Functions

In [17]:
function compute_mse(X, y, beta,beta0) #inc beta0
    n,p = size(X)
    beta_0= [beta0 for i = 1:n]
    
    return (sum((beta0.+Matrix(X)*beta .- Array(y)).^2)/n)
end

compute_mse (generic function with 1 method)

In [18]:
function compute_r2(X, y, beta,beta0) #inc beta0
    
    beta_0= [beta0 for i = 1:n]
    
    SSres = sum( (Array(y) .- Matrix(X)*beta.+beta0).^2 )
    SStot = sum( (y .- Statistics.mean(Array(y))).^2 )
    return 1-SSres/SStot
end

compute_r2 (generic function with 1 method)

# 3.a Robust Regression


$$\min_{\beta} \sum_{i=1}^n | y_i-\beta_0-\beta^Tx_i| + \lambda \sum_{i=0}^p |\beta_i|.$$

We rewrite the above problem as:
$$\min_{\beta,t,a} \quad \sum_{i=1}^n t_i + \lambda \sum_{j=0}^p a_j
\quad \text{s.t.} \\
\quad \ (y-\beta_0-\beta^Tx_i)\ \leq t_i,\\
-\quad \ (y-\beta_0-\beta^Tx_i)\ \leq t_i,\\
\quad \beta_j \leq a_j,\\
\quad -\beta_j \leq a_j.$$



In [19]:
using JuMP, Gurobi

In [20]:
model = Model(Gurobi.Optimizer)

Academic license - for non-commercial use only - expires 2022-08-18


A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi

# 1.3 a) Regression 1

In [21]:
rho_vals=[0.01, 0.03, 0.08, 0.1, 0.3, 0.8, 1, 3];

In [22]:
gurobi_env=Gurobi.Env()

function l1_regression(X, y, rho; solver_output=0)
    n,p = size(X)
    
    # Build model
    model = Model(with_optimizer(Gurobi.Optimizer,gurobi_env))
    set_optimizer_attribute(model, "OutputFlag", solver_output) 
    
    # Insert variables
    @variable(model,beta[j=1:p])
    @variable(model,a[j=1:p]>=0) 
    @variable(model, t[j=1:n]>=0) 
    @variable(model, d>=0)
    @variable(model, beta0) #
    
    #Insert constraints
    @constraint(model,[j=1:p], beta[j]<=a[j])
    @constraint(model,[j=1:p], -beta[j]<=a[j])
    @constraint(model, beta0<=d)
    @constraint(model, -beta0<=d)
    @constraint(model, [z=1:n],(((y[z,1]-beta0-transpose(beta)*Vector(X[z,:])))) <= t[z]) #X[z,:] 9 elements in row of X_z
    @constraint(model,[z=1:n], -(((y[z,1]-beta0-transpose(beta)*Vector(X[z,:])))) <= t[z])
    
    #Objective
    @objective(model,Min, sum(t[z] for z=1:n) + rho*(sum(a[j] for j=1:p)+d) )
    
    # Optimize
    optimize!(model)
    
    # Return estimated betas
    return (value.(beta0),value.(beta))
end

Academic license - for non-commercial use only - expires 2022-08-18


l1_regression (generic function with 1 method)

# 1.3b

### 1.3b i) For the data from the train and valid files, randomly split it into 70% training and 30% validation

In [23]:
function robust_regression_valid(X, y,seed, rho_vals; method=l1_regression, split_at=0.7, solver_output=0)
    n,p = size(X) #3132, 7
    Random.seed!(seed)
    split = convert(Int,floor(split_at*n)) #floor takes the integer part
    
    
    #To create train and validation data, we will define the indices of each data.
    permuted_indices = randperm(n)
    train_indices, valid_indices = permuted_indices[1:split], permuted_indices[split+1:end]

    X_train, y_train = X[train_indices,:], y[train_indices,:]
    
    X_valid, y_valid = X[valid_indices,:], y[valid_indices,:]

    #create an array to hold the results
    errors = zeros(length(rho_vals))
    
    for (i,rho) in enumerate(rho_vals)
        #get the beta coefficients from the l1_regression
        
        
        beta0,beta = method(X_train,y_train,rho,solver_output=solver_output)
        
        #compute the MSE with the optimal beta we just found
        errors[i] = compute_mse(X_valid, y_valid, beta,beta0)
    
    end
    
    #get the best performing rho
    
    i_best = argmin(errors)
    best_rho=rho_vals[i_best]
    
    #get the betas with best rho
    best_betas = method(X,y,best_rho)
    
    return best_betas,best_rho, errors
    
end

robust_regression_valid (generic function with 1 method)

### 1.3 b) ii) Use the validation set to select an optimal value for the parameter λ in the set {0.01, 0.03, 0.08, 0.1, 0.3, 0.8, 1, 3}.

In [33]:
beta_l1, rho_l1, errors_l1 = robust_regression_valid(stable_x,stable_y,1,rho_vals;method=l1_regression)

beta_l1=pushfirst!(beta_l1[2],beta_l1[1])

println("Number of zeroed features = ",(sum(abs.(beta_l1).<1e-4)))

#println("MSE: ", compute_mse(stableX_test,stableY_test,beta_l1[2:8],beta_l1[1]))

println("Optimal rho: ", rho_l1)

#println("Betas:",beta_l1)

Number of zeroed features = 17
Optimal rho: 3.0


### 1.3 b) iii) Try recombining the training and validation sets and then splitting them up in a different manner a few times (at least 5), and then training/validating a new model.

In [34]:
MSE_Test_Models=zeros(10)

p=10
for i =1:p
    #Recombines training and validation sets
    #Splits them differenetly with different seeds
    #Trains and validates a new model returning best betas from training set and rho from validation
    beta_l1_cv, rho_l1_cv, errors_l1 = robust_regression_valid(stable_x,stable_y,i,rho_vals;method=l1_regression)
    
    beta_l1_cv2=pushfirst!(beta_l1_cv[2],beta_l1_cv[1])
    
    println("Number of zeroed features for test model ", i ,": ",(sum(abs.(beta_l1_cv2).<1e-4)))
    
    #Calculate MSE on Test Set for best Rho in each of the 10 splits
    
    #MSE_Test_Model=compute_mse(stableX_test, stableY_test, beta_l1_cv2[2:8],beta_l1_cv2[1])
    
    #println("Validated lambda for test model ", i ,": ", rho_l1_cv)
    #println("MSE for test model ", i , ": ", MSE_Test_Model,[i])

    #MSE_Test_Models[i]=MSE_Test_Model
    
end

# range of (MSEs)  on the test data for each of these models

#n  println("Largest MSE from Test Models ",MSE_Test_Models[argmax(MSE_Test_Models)])
#println("Smallest MSE from Test Models ",MSE_Test_Models[argmin(MSE_Test_Models)])
#println("Average MSE from Test Models ",mean(MSE_Test_Models))

#println("Range for Test MSE: ", MSE_Test_Models[argmax(MSE_Test_Models)] - MSE_Test_Models[argmin(MSE_Test_Models)])



Number of zeroed features for test model 1: 17
Number of zeroed features for test model 2: 12
Number of zeroed features for test model 3: 17
Number of zeroed features for test model 4: 17
Number of zeroed features for test model 5: 17
Number of zeroed features for test model 6: 11
Number of zeroed features for test model 7: 12
Number of zeroed features for test model 8: 17
Number of zeroed features for test model 9: 12
Number of zeroed features for test model 10: 17


### 1.3 b) iii) What is the range of Mean Squared Errors (MSEs) you get on the test data for each of these models?

Largest MSE from Test Models 5.024122859285739

Smallest MSE from Test Models 4.88788585345981

Average MSE from Test Models 4.956380782543436

Range for Test MSE: 0.13623700582592857

# 3.1 c)

In [26]:
gurobi_env=Gurobi.Env()

function l2_regression2(X, y, rho,split_at; solver_output=0)
    n,p = size(X)
    
    
    k = convert(Int,floor(split_at*n)) #floor takes the integer part
    
    # Build model
    model = Model(with_optimizer(Gurobi.Optimizer,gurobi_env))
    set_optimizer_attribute(model, "OutputFlag", solver_output) 
    
    # Insert variables
    @variable(model,beta[j=1:p])
    @variable(model, beta0) 
    @variable(model,theta)
    
    @variable(model,a[j=1:p]) 
    @variable(model, u[i=1:n]>=0) 
    @variable(model, d>=0)
 
    
    #Insert constraints   
    @constraint(model,[j=1:p], beta[j]<=a[j])   
    @constraint(model,[j=1:p], -(beta[j])<=a[j])
    @constraint(model, [i=1:n], ( (y[i,1]-beta0-transpose(beta)*Vector(X[i,:])) ) <= theta+u[i])  
    @constraint(model,[i=1:n], -( (y[i,1]-beta0-transpose(beta)*Vector(X[i,:]) )) <= theta+u[i])
    @constraint(model,[i=1:n],u[i]>=0)
    @constraint(model, beta0<=d)
    @constraint(model, -beta0<=d) 
    
    #Objective
    @objective(model,Min, k*theta+sum(u[i] for i=1:n) + rho*(sum(a[j] for j=1:p)) )   
    
    # Optimize
    optimize!(model)
    
    # Return estimated betas
    beta0=value.(beta0)    
    beta=value.(beta)

    
    errors=zeros(n) 
    beta_0= [beta0 for i = 1:n]
       
    errors= ((beta0.+Matrix(X)*beta .- Array(y)).^2) .^0.5
    
    #Sort indices of MSE's
    indicies=sortperm(errors[:,1])
    
    #for (i,rho) in enumerate(indicies)
    #    print(i,rho)
    #end
    
    valid_indices=indicies[1:n-k] 
    training_indices=indicies[1+n-k:n]
      
    x_valid_set, y_valid_set = X[valid_indices,:], y[valid_indices,:]
    x_training_set,y_training_set=X[training_indices,:],y[training_indices,:]
   
    
    return (value.(beta0),value.(beta),errors,x_valid_set,y_valid_set,x_training_set,y_training_set)
end

Academic license - for non-commercial use only - expires 2022-08-18


l2_regression2 (generic function with 1 method)

# 3.d

### 3.d i) ii) iii) CREATING AND EXPORT TOUGHEST SUBSET | Project

In [44]:
#collect the toughest training set

beta0,beta,errors,x_valid_set,y_valid_set,x_training_set,y_training_set= l2_regression2(stable_x,stable_y,.01,.7);
x_training_set
y_training_set

In [50]:
x_training_set

,Item_Weight,Item_Visibility,Item_MRP,Years_Opened,Item_Type_Baking Goods,Item_Type_Breads
,Float64,Float64,Float64,Int64,Int64,Int64
1,7.155,0.168527,33.6874,11,0,1
2,12.35,0.0418025,34.8216,14,0,0
3,10.85,0.0480043,117.581,15,0,0
4,9.3,0.0422563,124.439,26,0,0
5,12.15,0.062276,37.9532,9,0,0
6,9.0,0.0692291,55.9614,11,0,0
7,16.75,0.0137113,100.267,15,0,0
8,12.7929,0.0977687,142.45,28,0,0
9,6.385,0.140328,109.16,15,0,0


In [51]:
y_training_set

4474×1 Matrix{Float64}:
  1023.3346
   311.5944
   351.5424
  1609.9044
  1078.596
   386.8298
   203.7348
   564.5984
   107.8596
  1070.6064
  1062.6168
   910.8144
  1609.9044
     ⋮
 10072.8882
  8994.958
  8323.8316
 10236.675
  9275.9256
 10306.584
  9779.9362
  9664.7528
 10993.6896
 11445.102
 12117.56
 13086.9648

In [48]:
using Tables
CSV.write( "x_training_stable.csv",  x_training_set)
CSV.write("y_training_stable.csv",  Tables.table(y_training_set), writeheader=false)

"y_training_stable.csv"

In [26]:
errors1 = zeros(length(rho_vals))
rho_errors= zeros(length(rho_vals))
MSE_list= zeros(length(rho_vals))



for (i,rho) in enumerate(rho_vals)
    beta0,beta,errors,x_valid_set,y_valid_set,x_training_set,y_training_set= l2_regression2(stable_x,stable_y,rho,.7)#,#points in training set)
    
    
    #Calculate MSE on validation set for each rho
    errors1[i] = compute_mse(x_valid_set,y_valid_set, beta,beta0)



end  

#MSE on validation set for each rho
for (i,rho) in enumerate(rho_vals)
    println("MSE for lambda ",rho, " on validation set: " ,errors1[i])

end

#Choose rho corresponding to the lowest MSE as your optimal 
println("Lambda with lowest MSE: " , rho_vals[argmin(errors1)])

#use the function from part (a) on  combined training/validation set with the optimal λ
beta0,beta=l1_regression(stable_x,stable_y, rho_vals[argmin(errors1)]; solver_output=0)

#MSE on model on test data

###FinalMSE = compute_mse(stableX_test, stableY_test, beta,beta0)

###println("MSE on test data using Lambda with lowest MSE from above: ", FinalMSE)

#print("rho & MSE:", errors) #why errors is bigger than just 1 mse for each lambda

MSE for lambda 0.01 on validation set: 35236.91965583911
MSE for lambda 0.03 on validation set: 35236.91965583911
MSE for lambda 0.08 on validation set: 35236.59924366356
MSE for lambda 0.1 on validation set: 35236.59924366356
MSE for lambda 0.3 on validation set: 35247.60705311133
MSE for lambda 0.8 on validation set: 35276.847146715314
MSE for lambda 1.0 on validation set: 35200.44920561531
MSE for lambda 3.0 on validation set: 35293.20320378044
Lambda with lowest MSE: 1.0


(-1213.918935098964, [0.8467673274327916, -48.971670709658014, 14.480589422280781, -13.959939921318218, -13.483067441165531, 31.426580567747123, -154.99498501070866, 0.0, -134.9223124231603, -74.28427140915664  …  -30.491275987267045, -16.51082468940278, -5.120080816921965, -6.467144518637137, 116.98484536202608, 0.0, 0.0, 4.637132924641932, 59.9593455826016, -51.40257179248158])

Lambda with lowest MSE: 0.1

MSE on test data using Lambda with lowest MSE from above: 4.891723552391911

### 1.3 d iv) Compare to your results from part (b) – how do the MSEs compare?

The MSE of this new model, taking the optimal lambda as 0.1, is 4.891723552391911.

Compared to the results in part b) where

The largest Mean Squared Error (MSE) from Test Models is 5.024122859285739

The smallest MSE from Test Models is 4.88788585345981

The range for Test MSE is 0.13623700582592857

We see that we are at the lower bound of the Mean Squared Errors for our test data.
        
Compared to the average MSE for the test set in part b of 4.956380782543436 (obtained after splitting the training and 
validation set in 10 different ways) we see that our final test MSE using this robust optimisation is lower than this mean also.
    
It may be the case that, given a different test set, repeating part b may give a different minimun test MSE which is 
slightly higher or lower than that given in part b, but our test MSE using this stable robust method should consistently be lower than the
mean test MSE obtained though finding lambda and betas from splitting the data randomly. This is another benefit of incoorporating robust stable regression.

Using stable regression for hyperparameter tuning produces more consistent optimal lambdas (0.01) than those produced
randomly splitting the data (0.1,0.03,0.1,0.3)

Interestingly, if we use the betas obtained from the robust model (that created the training set on the toughest 70% of training and validation 
data) that produces the lowest MSE with the associated lambda on the test data, we obtain a MSE of 4.854629988706049, which is 
slightly lower than that Test MSE obtained using the same optimal lambda and betas found using the training and validation set 
combined (see below).

In [70]:
#Using the optimal lambda (0.1) obtained above in the robust regression and applying the betas obtained on the test set

beta0,beta,errors,x_valid_set,y_valid_set,x_training_set,y_training_set= l2_regression2(stable_x,stable_y,0.1,.7)#,#points in training set)

FinalMSE_betas2 = compute_mse(stableX_test, stableY_test, beta,beta0)

println("MSE on test data using optimal Lambda (0.1) and betas from toughest training set: ", FinalMSE_betas2)


MSE on test data using optimal Lambda (0.1) and betas from toughest training set: 4.854629988706049
